## Output PDB with Neff or entropy as B factors

Want to color 229E spike based on site variability.

In [1]:
import collections
import copy
import os
import warnings

import Bio.PDB

import dms_variants.pdb_utils

from IPython.display import display, HTML

import pandas as pd

In [2]:
pdbs_dir = './results/pdbs/pdb_outputs'
os.makedirs(pdbs_dir, exist_ok=True)

In [3]:
site_var_df = pd.read_csv('./results/variation_analysis/site_variability.csv')

In [4]:
display(HTML(site_var_df.head().to_html(index=False)))

site,label_site,wildtype,mutation,condition,protein_chain,protein_site,site_n_effective_amino_acids,site_entropy,mut_aa_frequency
1,1,M,-,229E_alignment,A B C,NaN,1.0,0.0,0.0
1,1,M,A,229E_alignment,A B C,NaN,1.0,0.0,0.0
1,1,M,C,229E_alignment,A B C,NaN,1.0,0.0,0.0
1,1,M,D,229E_alignment,A B C,NaN,1.0,0.0,0.0
1,1,M,E,229E_alignment,A B C,NaN,1.0,0.0,0.0


In [5]:
entropy_df = site_var_df[['site', 'protein_chain', 'site_entropy']]\
                        .rename(columns={'protein_chain': 'chain', 'site_entropy': 'metric'})\
                        .drop_duplicates()
for chain in entropy_df['chain'].iloc[0].split(' '):
    entropy_df[chain] = entropy_df['metric']
entropy_df.drop(['chain', 'metric'], axis=1, inplace=True)
entropy_df = pd.melt(entropy_df, id_vars=['site'], var_name='chain', value_name='metric')
display(HTML(entropy_df.head().to_html(index=False)))

site,chain,metric
1,A,0.0
2,A,0.0
3,A,0.0
4,A,0.0
5,A,0.0


In [6]:
neff_df = site_var_df[['site', 'protein_chain', 'site_n_effective_amino_acids']]\
                        .rename(columns={'protein_chain': 'chain', 'site_n_effective_amino_acids': 'metric'})\
                        .drop_duplicates()
for chain in neff_df['chain'].iloc[0].split(' '):
    neff_df[chain] = neff_df['metric']
neff_df.drop(['chain', 'metric'], axis=1, inplace=True)
neff_df = pd.melt(neff_df, id_vars=['site'], var_name='chain', value_name='metric')
display(HTML(neff_df.head().to_html(index=False)))

site,chain,metric
1,A,1.0
2,A,1.0
3,A,1.0
4,A,1.0
5,A,1.0


In [7]:
original_pdbfile = './results/pdbs/6u7h.pdb'
reassigned_pdbfile_entropy = os.path.join(pdbs_dir, 'entropy_reassigned.pdb')
entropy_missing_metric=-1
dms_variants.pdb_utils.reassign_b_factor(input_pdbfile=original_pdbfile,
                                         output_pdbfile=reassigned_pdbfile_entropy,
                                         df=entropy_df,
                                         metric_col='metric',
                                         missing_metric=entropy_missing_metric)

In [8]:
reassigned_pdbfile_neff = os.path.join(pdbs_dir, 'neff_reassigned.pdb')
neff_missing_metric = 0
dms_variants.pdb_utils.reassign_b_factor(input_pdbfile=original_pdbfile,
                                         output_pdbfile=reassigned_pdbfile_neff,
                                         df=neff_df,
                                         metric_col='metric',
                                         missing_metric=neff_missing_metric)

## Look at missing sites

Some sites are not present in the pdb file. I want to see if these are variable. 

I am only looking at sites that are missing from the ectodomain. I do not care about the signal peptide or sites missing from the transmembrane or cytoplasmic tail domains. 

In [9]:
# missing_sites defined from pdb validation file
missing_sites = [*range(149, 163), *range(566, 575), *range(759, 768)]
print(missing_sites)

[149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 566, 567, 568, 569, 570, 571, 572, 573, 574, 759, 760, 761, 762, 763, 764, 765, 766, 767]


In [10]:
display(HTML(entropy_df[(entropy_df['chain']=='A') &
                        (entropy_df['site'].isin(missing_sites))].to_html(index=False)))

site,chain,metric
149,A,0.00000
150,A,0.00000
151,A,0.00000
152,A,0.00000
153,A,0.00000
154,A,0.00000
155,A,0.12742
156,A,0.00000
157,A,0.00000
158,A,0.00000


In [11]:
display(HTML(neff_df[(neff_df['chain']=='A') &
                     (neff_df['site'].isin(missing_sites))].to_html(index=False)))

site,chain,metric
149,A,1.00000
150,A,1.00000
151,A,1.00000
152,A,1.00000
153,A,1.00000
154,A,1.00000
155,A,1.09234
156,A,1.00000
157,A,1.00000
158,A,1.00000
